In [ ]:
pip install -r requirements.txt

# Import


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

import charset_normalizer
import fuzzywuzzy
from fuzzywuzzy import process
import os
import re
from dateutil import parser

# from scipy import stats
# from tabulate import tabulate
# import sklearn
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from sklearn.neighbors import NearestNeighbors
# from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
# from sklearn.metrics import silhouette_score, davies_bouldin_score,calinski_harabasz_score
# from yellowbrick.cluster import KElbowVisualizer
# from scipy.cluster.hierarchy import dendrogram, linkage

import warnings

warnings.filterwarnings("ignore")
plt.style.use(
    "https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle"
)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Character encoding


In [2]:
df_name = []
encoding = []
path = []

dataset_folder = "dataset"
for file in os.listdir(dataset_folder):
    file_path = os.path.join(dataset_folder, file)
    path.append(file_path)
    # look at the first ten thousand bytes to guess the character encoding
    with open(file_path, "rb") as rawdata:
        result = charset_normalizer.detect(rawdata.read())
        encoding.append(result["encoding"])
    # check what the character encoding might be
    file_name = os.path.splitext(file)[0]
    df_name.append(file_name)
    print(f"{file_name}: {result}")

context: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
mobile_plan_attr: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
mobile_plan_user: {'encoding': 'ascii', 'language': 'English', 'confidence': 1.0}
user: {'encoding': 'utf-8', 'language': 'English', 'confidence': 1.0}


In [3]:
data_source = (tuple(df_name), tuple(encoding), tuple(path))

In [4]:
context = pd.read_csv(data_source[2][0], encoding=data_source[1][0])
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,direction
1051,11420,Travel,&!Fri?end(s),?S&?unny,6 PM,1,0,0,0,1,20,0
6850,9842,Work,Alone,Sunny,6 AM,1,0,0,1,0,0,1
5856,8686,Visit,Alo% ne,!S~&&unny,18:00,1,1,0,1,0,67,1
4278,7064,Visit,Alone,Rainy,20:00,1,0,0,1,0,0,1
8592,15370,Travel,%?%Family,Sunn?y,1 PM,1,0,0,0,1,13,0


In [5]:
mobile_plan_attr = pd.read_csv(
    data_source[2][1], encoding=data_source[1][1], delimiter=";"
)

mobile_plan_attr.sample(5)

,mobile_plan,description,price,duration
0,DATASILVER,"2GB/ day, high speed",100000,5d
1,DATAGOLD,"5GB/ day, high speed",200000,5d
3,SOCIALMEDIAGOLD,"3GB/ day, high speed\nUnlimited for Tik Tok, F...",250000,3d
2,SOCIALMEDIA,"1GB/ day\nUnlimited for Tik Tok, Facebook, You...",150000,3d
4,DATACALL,"2GB/ day, high speed\n300 mins call for extern...",200000,5d


In [6]:
mobile_plan_user = pd.read_csv(data_source[2][2], encoding=data_source[1][2])
mobile_plan_user.sample(5)

,id,mobile_plan,accept
38487,8552.0,NaN,0.0
25874,8303.0,DATAGOLD,1.0
15589,4151.0,DATASILVER,0.0
6710,1939.0,SOCIALMEDIA,0.0
2897,13319.0,SOCIALMEDIA,1.0


In [32]:
user = pd.read_csv(data_source[2][3], encoding=data_source[1][3])
user.sample(5)

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,insta_freq,use_less_than_2GB,use_2GB_to_4GB
9306,5541,William Vasquez,Male,32,Masters,Education&Training&Library,1840000000 VND,M ar ried_0,US,+1-891-714-2178x5400,"Restaurant manager, fast food",1,0,4,8,0
5616,18457,Linda Fowler,Female,34,Highschool,Healthcare Support,777500000 VND,S i ngle_0,ENGLAND,707.517.2251,Arts development officer,0,3,1,1,0
8264,1199,Igor Ramos,Male,19,NaN,Student,20200$,Unmarr ied_0,BRAZIL,41 3452 2418,Geofísico,0,16,5,1,0
5645,4526,Jeffrey Collins,Male,22,NaN,Sales & Related,1160000000 VND,S i ngle_0,ENGLAND,001-615-341-5328,"Teacher, primary school",3,17,8,1,1
5528,5425,藤井 和也,Male,29,Highschool,Installation Maintenance & Repair,415000000 VND,Singl e_1,JAPAN,070-6719-4854,公認会計士,7,3,2,6,5


In [8]:
dataset = [context, mobile_plan_attr, mobile_plan_user, user]

# Cleaning
## Missing values check

In [9]:
for x in dataset:
  print('-----------------')
  print(x.isna().sum())

-----------------
id              0
purpose         0
go_with         0
weather         0
time            0
viettel_no_0    0
viettel_no_1    0
viettel_no_2    0
to_hanoi        0
to_other        0
score           0
direction       0
dtype: int64
-----------------
mobile_plan    0
description    0
price          0
duration       0
dtype: int64
-----------------
id                0
mobile_plan    2715
accept         2721
dtype: int64
-----------------
id                      0
name                    0
gender                  0
age                     0
education            4003
profession              0
income                  0
living_with             0
nation                  0
phone                   0
job                     0
fb_freq                 0
yt_freq                 0
insta_freq              0
use_less_than_2GB       0
use_2GB_to_4GB          0
dtype: int64


In [11]:
mobile_plan_user = mobile_plan_user.dropna(subset=['mobile_plan', 'accept'])

## Context

In [12]:
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,to_hanoi,to_other,score,direction
3547,15358,Travel,Frien%d~(s),R?%a~iny,6PM,1,0,0,0,1,0,0
154,13364,Travel,Fr%?ie%nd(s),Sunny,11AM,1,1,0,0,1,0,0
1773,10865,Travel,Fri!e n %d(s),!Su~nny,13:00,1,1,0,0,1,26,0
5782,10367,Work,Alone,Snowy,6:00,1,1,1,0,1,0,0
4007,6393,Travel,Family,&R!%~ainy,8:00,1,0,0,0,1,0,0


In [13]:
for x in ["go_with", "weather"]:
    # Eliminate special characters and white spaces
    context[x] = context[x].apply(lambda x: re.sub(r"[^a-zA-Z\s]", "", x))

    # Lowercase all words
    context[x] = context[x].str.lower()

    print(context[x].unique())

['alone' 'friends' 'fa mily' 'family' 'fami ly' 'a l one' 'fr iends'
 ' alone' 'a lone' '  alone' ' friends' 'f riends' 'f rie nds' 'frie nds'
 'frien ds' 'f amily' ' family' 'alo ne' 'friend s' 'al one' 'fri ends'
 ' a lone' ' fri ends' 'f riend s' 'alon e' 'frien  ds' ' fri e nds'
 'friends ' 'a  lone' 'fa  mily' 'al  one' '   alone' 'fri e nds'
 ' al one' 'fam ily' '  family' 'f rien ds' '  a lone' ' f amily'
 'frie   nds' 'fri en ds' 'f a mily' ' frie nds' 'f  amily' 'fr  iends'
 ' fa mily' 'famil y' ' fr iends' 'fr ie nds' 'fr ien ds' ' f  amily'
 'fam  ily' ' frien ds' 'frie nd s' 'frie n ds' ' friend s' 'frie  nds'
 'fr i ends' 'fri  ends' 'f ri ends' ' a  lone' ' alo ne' 'friend  s'
 'a   lone' '  friends' '   friends' ' fam ily' 'f r iends' 'alo  ne'
 '  a  lone' ' fa  mily' 'f r  iends' 'f  riends' ' frie  nds' 'fri end s'
 ' fami ly' 'f rie  nds' ' f riends' ' al  one' 'f a  mily' ' fa m ily'
 '    alone' 'f  ri ends' 'a lo ne' 'frien d s' 'f am ily' '  f riends'
 '   family

In [14]:
# List of possible values for the column 'go_with'
choices = ["alone", "friend(s)", "family"]


# Apply the fuzzy matching to the column 'go_with'
def correct_name(name):
    return process.extractOne(name, choices)[0]


context["go_with"] = context["go_with"].apply(correct_name)

In [15]:
# Eliminate white spaces
context["time"] = context["time"].str.replace(r"\s+", "", regex=True)


# Function to convert time to 24-hour format
def convert_to_24hr_format(time_str):
    # Use dateutil.parser to parse the time string
    return parser.parse(time_str).strftime("%H:%M")


# Apply the function to the column 'time'
context["time"] = context["time"].apply(convert_to_24hr_format)
context["time"] = pd.to_datetime(context["time"], format="%H:%M").dt.time

In [16]:
context["purpose"] = context["purpose"].str.strip()
context["purpose"] = context["purpose"].str.lower()

In [17]:
context.purpose.unique()

array(['travel', 'visit', 'work'], dtype=object)

In [18]:
context["purpose"] = context["purpose"].replace('travel', 'visit')

In [21]:
context = context.drop(axis=1, columns=["to_hanoi", "to_other"])

In [22]:
context.sample(5)

,id,purpose,go_with,weather,time,viettel_no_0,viettel_no_1,viettel_no_2,score,direction
982,11732,visit,friend(s),sunny,08:00:00,1,0,0,0,0
2964,3454,visit,family,sunny,18:00:00,1,1,0,0,0
5510,13239,visit,alone,sunny,22:00:00,1,1,0,6,0
849,19883,visit,friend(s),sunny,13:00:00,1,1,0,59,0
3491,5354,work,alone,sunny,07:00:00,1,0,0,0,1


## User


In [23]:
user.sample(5)

,id,name,gender,age,education,profession,income,living_with,nation,phone,job,fb_freq,yt_freq,insta_freq,use_less_than_2GB,use_2GB_to_4GB
7873,7996,Zachary Dawson,Male,67,NaN,Protective Service,1225000000 VND,Married_0,ENGLAND,9937476702,"Editor, commissioning",0,1,23,10,3
11139,2619,Анжелика Константиновна Владимирова,Female,64,Highschool,Sales & Related,1177500000 VND,Married_4,RUSSIA,8 (371) 889-07-18,Квасник,0,23,4,3,1
5715,10465,Matthew Walker,Male,32,Bachelor,Computer & Mathematical,77700$,Married_1,ENGLAND,(409)321-6647x31639,"Radiographer, diagnostic",3,0,1,1,1
1113,17251,정선영,Female,28,NaN,Management,657500000 VND,Ma rried_2,KOREA,054-929-8366,자동차 정비원,1,2,3,7,3
5541,12740,Michele Rodriguez,Female,35,NaN,Food Preparation & Serving Related,535000000 VND,Divorced_4,ENGLAND,577-804-5500x310,Farm manager,0,0,5,14,12


In [33]:
# Eliminate white spaces
user["living_with"] = user["living_with"].str.strip()
user["living_with"] = user["living_with"].str.replace(r"\s+", "", regex=True)
user["living_with"] = user["living_with"].str.lower()

In [34]:
user["nation"] = user["nation"].str.strip()
user["nation"] = user["nation"].str.lower()

In [35]:
user["gender"] = user["gender"].str.strip()
user["gender"] = user["gender"].str.lower()

In [36]:
user["education"] = user["education"].str.strip()
user["education"] = user["education"].str.lower()

In [37]:
user.nation.unique()

array(['australia', 'england', 'korea', 'denmark', 'brazil', 'japan',
       'us', 'china', 'russia'], dtype=object)

In [38]:
user["nation"] = user["nation"].replace("australia", "oceania")
user["nation"] = user["nation"].replace(["brazil", "us"], "americas")
user["nation"] = user["nation"].replace(["denmark", "england", "russia"], "europe")
user["nation"] = user["nation"].replace(["korea", "japan", "china"], "asia")

In [40]:
user["nation"] = user["nation"].rename("continent")

In [41]:
user["education"] = user["education"].fillna("unknown")
user["education"] = user["education"].replace("bachelor", "grad")
user["education"] = user["education"].replace(["associate", "highschool"], "undergrad")
user["education"] = user["education"].replace("masters", "postgrad")

In [42]:
user.profession.unique()

array(['Unemployed', 'Architecture & Engineering', 'Student',
       'Education&Training&Library', 'Healthcare Support',
       'Healthcare Practitioners & Technical', 'Sales & Related',
       'Management', 'Arts Design Entertainment Sports & Media',
       'Computer & Mathematical', 'Life Physical Social Science',
       'Personal Care & Service', 'Community & Social Services',
       'Office & Administrative Support', 'Construction & Extraction',
       'Legal', 'Retired', 'Installation Maintenance & Repair',
       'Transportation & Material Moving', 'Business & Financial',
       'Protective Service', 'Food Preparation & Serving Related',
       'Production Occupations',
       'Building & Grounds Cleaning & Maintenance',
       'Farming Fishing & Forestry'], dtype=object)

In [30]:
user.living_with.unique()

array(['unmarried_2', 'unmarried_1', 'single_0', 'married_1', 'married_2',
       'married_4', 'married_3', 'unmarried_0', 'single_2', 'single_1',
       'single_4', 'single_3', 'divorced_2', 'divorced_1', 'divorced_3',
       'married_0', 'unmarried_4', 'divorced_4', 'widowed_0',
       'unmarried_3', 'widowed_4', 'widowed_1', 'widowed_3', 'widowed_2',
       'divorced_0'], dtype=object)

In [43]:
# Seperate Living_With column into Maritual_Status column and Children column
user[["marital_status", "children"]] = user["living_with"].str.split("_", expand=True)


user["marital_status"] = user["marital_status"].apply(
    lambda x: "relationship" if x == "married" else "single"
)

## mobile_plan_attr

In [46]:
mobile_plan_attr.head()

,mobile_plan,description,price,duration
0,DATASILVER,"2GB/ day, high speed",100000,5d
1,DATAGOLD,"5GB/ day, high speed",200000,5d
2,SOCIALMEDIA,"1GB/ day\nUnlimited for Tik Tok, Facebook, You...",150000,3d
3,SOCIALMEDIAGOLD,"3GB/ day, high speed\nUnlimited for Tik Tok, F...",250000,3d
4,DATACALL,"2GB/ day, high speed\n300 mins call for extern...",200000,5d


In [47]:
mobile_plan_attr['duration'] = mobile_plan_attr['duration'].str.replace('d', '')

In [48]:
mobile_plan_attr['capacity'] = mobile_plan_attr['description'].str.extract(r'(\d+)GB')

In [49]:
mobile_plan_attr.head()

,mobile_plan,description,price,duration,capacity
0,DATASILVER,"2GB/ day, high speed",100000,5,2
1,DATAGOLD,"5GB/ day, high speed",200000,5,5
2,SOCIALMEDIA,"1GB/ day\nUnlimited for Tik Tok, Facebook, You...",150000,3,1
3,SOCIALMEDIAGOLD,"3GB/ day, high speed\nUnlimited for Tik Tok, F...",250000,3,3
4,DATACALL,"2GB/ day, high speed\n300 mins call for extern...",200000,5,2


## mobile_plan_user

In [50]:
mobile_plan_user.head()

,id,mobile_plan,accept
0,11156.0,DATASILVER,1.0
1,4297.0,SOCIALMEDIAGOLD,1.0
2,13301.0,DATASILVER,1.0
3,9920.0,SOCIALMEDIAGOLD,1.0
4,8424.0,DATASILVER,1.0


In [53]:
mobile_planes = mobile_plan_user.mobile_plan.unique()

In [54]:
mobile_plan_user['mobile_plan'] = mobile_plan_user['mobile_plan'].str.lower()

### Mobile plan acceptance

In [55]:
# Sử dụng pivot_table để tổng hợp dữ liệu
mobile_plan_accept = mobile_plan_user.pivot_table(
    index="id", columns="mobile_plan", values="accept", aggfunc="sum", fill_value=0
).reset_index()

# Đảm bảo tất cả các cột trong mobile_planes có mặt trong bảng kết quả cuối cùng
for plan in mobile_planes:
    if plan not in mobile_plan_accept.columns:
        mobile_plan_accept[plan] = 0

# Sắp xếp lại các cột theo thứ tự trong mobile_planes
mobile_plan_accept = mobile_plan_accept[["id"] + list(mobile_planes)]

In [56]:
mobile_plan_accept

mobile_plan,id,datasilver,socialmediagold,socialmedia,datacall,datagold
0,1000.0,1.0,0.0,0.0,0.0,0.0
1,1001.0,0.0,0.0,1.0,0.0,0.0
2,1002.0,0.0,1.0,0.0,0.0,0.0
3,1004.0,0.0,0.0,0.0,0.0,0.0
4,1005.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
11567,19994.0,0.0,0.0,0.0,0.0,0.0
11568,19996.0,0.0,0.0,0.0,1.0,0.0
11569,19997.0,1.0,0.0,0.0,0.0,0.0
11570,19998.0,0.0,0.0,0.0,0.0,0.0


### Mobile plan recommendation

In [57]:
mobile_plan_rec_raw = mobile_plan_user[["id"]]

# Create columns for each item in the mobile_plan_user dataframe
for item in mobile_planes:
    mobile_plan_rec_raw[item] = mobile_plan_user["mobile_plan"].apply(
        lambda x: 1 if x == item else np.nan
    )

# Check rows that have duplicated ID
duplicate_rows = mobile_plan_rec_raw[
    mobile_plan_rec_raw.duplicated(subset=["id"], keep=False)
]
# Sort by ID
duplicate_rows.sort_values("id")

# Fill missing values of each ID group
# In each ID group, each missing value will be replaced by the last valid value forward (ffill), otherwise, by the next valid value backward (bfill).
mobile_plan_rec = duplicate_rows.groupby("id").apply(
    lambda x: x.fillna(method="ffill").fillna(method="bfill")
)
mobile_plan_rec = mobile_plan_rec.drop_duplicates()

mobile_plan_rec = mobile_plan_rec.drop(axis=1, columns="id").reset_index()
mobile_plan_rec = mobile_plan_rec.drop(axis=1, columns="level_1")

mobile_plan_rec = mobile_plan_rec.fillna(0)
mobile_plan_rec

,id,datasilver,socialmediagold,socialmedia,datacall,datagold
0,1000.0,1.0,0.0,1.0,1.0,0.0
1,1001.0,0.0,0.0,1.0,1.0,0.0
2,1005.0,0.0,0.0,1.0,1.0,1.0
3,1006.0,0.0,0.0,1.0,1.0,0.0
4,1008.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...
10107,19992.0,0.0,1.0,1.0,1.0,0.0
10108,19996.0,0.0,1.0,1.0,1.0,1.0
10109,19997.0,1.0,1.0,0.0,0.0,1.0
10110,19998.0,1.0,1.0,1.0,0.0,0.0


# Data type check


In [59]:
dataset = [context, mobile_plan_attr, mobile_plan_accept, mobile_plan_rec, user]
for x in dataset:
    print("-------------------------")
    print(x.dtypes)

-------------------------
id               int64
purpose         object
go_with         object
weather         object
time            object
viettel_no_0     int64
viettel_no_1     int64
viettel_no_2     int64
score            int64
direction        int64
dtype: object
-------------------------
mobile_plan    object
description    object
price           int64
duration       object
capacity       object
dtype: object
-------------------------
mobile_plan
id                 float64
datasilver         float64
socialmediagold    float64
socialmedia        float64
datacall           float64
datagold           float64
dtype: object
-------------------------
id                 float64
datasilver         float64
socialmediagold    float64
socialmedia        float64
datacall           float64
datagold           float64
dtype: object
-------------------------
id                    int64
name                 object
gender               object
age                   int64
education            objec

In [62]:
mobile_plan_attr.duration = mobile_plan_attr.duration.astype(int)
mobile_plan_attr.capacity = mobile_plan_attr.capacity.astype(int)

mobile_plan_accept.id = mobile_plan_accept.id.astype(int)
mobile_plan_accept.datasilver = mobile_plan_accept.datasilver.astype(int)
mobile_plan_accept.socialmediagold = mobile_plan_accept.socialmediagold.astype(int)
mobile_plan_accept.socialmedia = mobile_plan_accept.socialmedia.astype(int)
mobile_plan_accept.datacall = mobile_plan_accept.datacall.astype(int)
mobile_plan_accept.datagold = mobile_plan_accept.datagold.astype(int)

mobile_plan_rec.id = mobile_plan_rec.id.astype(int)
mobile_plan_rec.datasilver = mobile_plan_rec.datasilver.astype(int)
mobile_plan_rec.socialmediagold = mobile_plan_rec.socialmediagold.astype(int)
mobile_plan_rec.socialmedia = mobile_plan_rec.socialmedia.astype(int)
mobile_plan_rec.datacall = mobile_plan_rec.datacall.astype(int)
mobile_plan_rec.datagold = mobile_plan_rec.datagold.astype(int)

user.children = user.children.astype(int)